In [ ]:
############ This file was used to create Generate_Forecast.jl. There have also been changes to that file so this is not a good starting place. 

In [1]:
#### Straight from Kyle's Code
## Step by step
#+++Define T, G, ncust, obs, nreal++++
# Below is what is done in the file
# Generate historical demand for all customers.
# Classify what the historical demand is on the quad chart
# Use appropriate forecasting method to generate forecast for time horiz T
# Generate demand realizations off of parameters used to create forecast
# (used in model performance evlauation)
# Determine little s in (s,S) policy using markov bootstrap technique.
# Feed forecast to MILP for optimal policy determination


#Test Conditions
avg_dem = 3
adi = 1.16
cv2 = 0.24
lev = avg_dem * adi
T = 60
G = 3
ncust = 2
nsup = 2
obs = 180
nreal = 50 # # realizations to generate
LT = 7 #based on LT of resupply from TM to supplier
N=1000 # Number of replications to generate LTD based on

1000

In [2]:
##needed referred files
using Plots
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/3. Functions/simID_function/simID.jl")
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/3. Functions/simID_function/idClass.jl")
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/3. Functions/Forecasting_Functions/CrostonSBA.jl")
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/3. Functions/Forecasting_Functions/Croston.jl")
include("/Users/Matthew/Documents/Grad School/Thesis Work/Julia_Code/3. Functions/Forecasting_Functions/MarkovBootstrap.jl")

MarkovBootstrap

In [3]:
#Packages needed
using Optim
using Plots
using StatsBase
using Distributions
using Statistics
using Random

In [4]:
####### Gen_historical but with optional number of customers and productucts
"""
    Generate historical demand per product for each customer. 
Inputs:
    adi: 
    CV2: 
    lev: 
    G: Number of products
    ncust: Number of customers
    obs: Number of time periods of historical data to generate
Outputs: 
    (ncust x obs x G) array of historical demand
"""
function gen_historical(adi, cv2, lev, G, ncust, obs=180)
    if G < 1 
        error("ERROR: G is less than 1, pick value of 1 or more")
    end
    if ncust < 0
        error("ERROR: ncust is less than 1, pick value of 1 or more")
    end
    
    historical = Array{Int64}(undef,ncust,obs,G)
    
    for g in 1:G #iterate for number of products
        for c in 1:ncust #iterate for number of customers
            historical[c,:,g] = simID(1,obs,adi,cv2,lev)
        end
    end
    return historical
end

gen_historical

In [5]:
"""
Generate demand realizations based off original input parameters (to be used in EVAL)
Inputs:
    nreal: number of realizations: 1st dimension
    adi:
    cv2:
    lev:
    T: Time horizon to be evaluated: dimension
    G: Number of products: 4th dimension
    ncust: Number of customers: 3rd dimension
Outputs:
""" ###MIGHT NEED TO EDIT THIS SLIGHTLY
function gen_dem_realization(nreal, adi, cv2, lev, T, G, ncust)
    obs = T
    cust_dem = Array{Int64}(undef, nreal, obs, ncust, G) 
    for g in 1:G
        for c in 1:ncust
            cust_dem[:,:,c,g] = simID(nreal,obs,adi,cv2,lev)
        end
    end
    return cust_dem
end

gen_dem_realization

In [6]:
"""
Function that first takes historical demand then classifies it and then gives a forcast
Inputs:
    hist_dem: historical demand that should already be generated
    G: number of Customers
    T: Time horizon interested in predicting
Outputs:
    A forecast of future demand
"""
function forecast_dem(hist_dem,G,T)
    #loop through each product type of all customers
    forecast = Array{Int64}(undef, size(hist_dem)[1],T,G)    ######might need to change this later as model changes
    for g in 1:G
        data = hist_dem[:,:,g]
        ## Classify demand and determine forecasting method
        # Outputs idx of demand streams that need to be forecasted by whatever method
        idx_SBA = idClass(data)[7]
        idx_Croston = idClass(data)[8]
        @show(idx_Croston)
        @show(idx_SBA)
        
        ## Forecast SBA and Croston
        # define function handle used to determine alpha value that minimizes mse
        mseh(x,y) = mean(filter(!isnan, (x .- y).^2)) # mse fh. x: historical data, y: forecasted value output.
        # set starting alpha value
        a = [0.1]
        F =  Array{Float64}(undef, (length(idx_SBA) + length(idx_Croston)))
        aoptSBA =Array{Float64}(undef, length(idx_SBA))
        aoptCrost = Array{Float64}(undef, length(idx_Croston))
        
        
        if !isempty(idx_SBA) #checker to ensure idx_SBA has values in it
            for i = idx_SBA # idx of customer demand for product G that requires SBA forecast
                if i != 0 # only do this for nonzero values
                    # Objective is to minimize the mse by optimizing alpha value
                    temp_aoptSBA = Optim.optimize(alpha -> mseh(data[i,:], CrostonSBA(data[i,:], T, alpha)[2]), a, Newton()).minimizer # minimize mse by changing alpha value
                    aoptSBA[i] = temp_aoptSBA # save optimal alpha value for checking
                    F[i,1] = CrostonSBA(data[i,:],T,aoptSBA[i])[1] # Run function using the optimized alpha value
                end
            i # checking
            end
        end
        
        if !isempty(idx_Croston) #checker to ensure idx_Croston has values in it
            for i = idx_Croston # idx of customer demand for product G that requires SBA forecast
                if i != 0 
                    #min mse by changing alpha value
                    temp_aoptCrost = Optim.optimize(alpha -> mseh(data[i,:], Croston(data[i,:],T,alpha)[2]), a, Newton()).minimizer
                    aoptCrost[i] = temp_aoptCrost[1] # save optimal alpha value for checking
                    F[i,1] = Croston(data[i,:],T,aoptCrost[i])[1]
                end
                #i # checking
            end
        end
        #Forecast for time horizon
        forecast[:,:,g] .= ceil.(F)
    end
    return(forecast)
end

forecast_dem

In [7]:
###### Determining the min policy code
#Markov Bootstrap code for little s inventory policy
"""
Inputs:
    nsup: number of suppliers
    ncust: number of customers
    LT: based on LT of resupply from TM to the supplier
    rep: Number of replications to generate LTD based on
    hist_dem: historical demand calculated earlier
Outputs:
    LTD - Lead time duration for each product
    Q - Markov bootstrap for each product 75th percentile  ###### better description
    pAll - little s for each product (nsup x G matrix)
"""
function little_s_inventory(nsup, ncust, LT, rep, hist_dem)
    historical_byprodcut = sum(hist_dem,dims=1) # forecast based on all customers combined. Look at each product seperatley
    
    #determine LTD (Q) for each product
    G = size(hist_dem)[3]
    
    #initiate arrays
    Q = zeros(G,1)
    LTD = zeros(G,rep)
    for g = 1:G
        (LTD[g,:],Q[g,1]) = MarkovBootstrap(historical_byprodcut[:,:,g],LT,rep) #ltd PER PRODUCT
    end
    Q = round.(Q) #this is by product for the entire system. Divide it up between all sups
    
    z = round.(Q/nsup)
    pAll = zeros(nsup,G)
    for g in 1:G
        pAll[:,g] = repeat([z[g]],nsup,1)
    end
    
    return (LTD, Q, pAll)
    
end

little_s_inventory

In [1]:
function AM_design(nsup,ncust)
    q = []
    for i in 0:2^(nsup+ncust)-1
        push!(q,string(i,base=2,pad=nsup+ncust))
    end
    NetDesign = zeros(2^(nsup+ncust),nsup+ncust)
    for i in 1:length(q)
        for j in 1:length(q[i])
            NetDesign[i,j] = parse(Int8,q[i][j])
        end
    end
    return NetDesign # All suppliers are first, customers are last.
end

AM_design (generic function with 1 method)

In [14]:
hist_dem = gen_historical(adi, cv2, lev, G, ncust)
dem_real = gen_dem_realization(nreal, adi, cv2, lev, T, G, ncust);
forecast = forecast_dem(hist_dem,G,T)
(LTD, Q, pAll) = little_s_inventory(nsup, ncust, LT, N, hist_dem)
NetDesign = AM_design(nsup,ncust)
ndesign = size(NetDesign)[1]

idx_Croston = [1, 2]
idx_SBA = Int64[]
idx_Croston = [1, 2]
idx_SBA = Int64[]
idx_Croston = [1, 2]
idx_SBA = Int64[]


16